#  Instrusion Detection System

In [1]:
#Libraries

import numpy as np
import pandas as pd
import joblib

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# to display all column of datapoints
pd.set_option('display.max_columns', None)

In [2]:
df_test=pd.read_csv("cleaned_test.csv")

df_test.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class_label
0,0,udp,domain_u,SF,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.0,0.0,0.5,1.0,0.00,10,3,0.30,0.30,0.30,0.00,0.00,0.0,0.00,0.00,normal.
1,0,tcp,http,SF,223,185,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.00,71,255,1.00,0.00,0.01,0.01,0.00,0.0,0.00,0.00,normal.
2,0,tcp,http,SF,230,260,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,19,0.0,0.0,0.0,0.0,1.0,0.0,0.11,3,255,1.00,0.00,0.33,0.07,0.33,0.0,0.00,0.00,normal.
3,1,tcp,smtp,SF,3170,329,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.00,54,39,0.72,0.11,0.02,0.00,0.02,0.0,0.09,0.13,normal.
4,0,tcp,http,SF,297,13787,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,177,255,1.00,0.00,0.01,0.01,0.00,0.0,0.00,0.00,normal.


In [3]:
def data_preprocessing(X):
    
    """
    Normal=0 ,Probing=1,DOS=2,U2R=3,R2L=4
    
    """
    
    mapping = {'ipsweep.': 1,'satan.': 1,'nmap.': 1,'portsweep.': 1,'saint.': 1,'mscan.': 1,
        'teardrop.': 2,'pod.': 2,'land.': 2,'back.': 2,'neptune.': 2,'smurf.': 2,'mailbomb.': 2,
        'udpstorm.': 2,'apache2.': 2,'processtable.': 2,
        'perl.': 3,'loadmodule.': 3,'rootkit.': 3,'buffer_overflow.': 3,'xterm.': 3,'ps.': 3,
        'sqlattack.': 3,'httptunnel.': 3,
        'ftp_write.': 4,'phf.': 4,'guess_passwd.': 4,'warezmaster.': 4,'warezclient.': 4,'imap.': 4,
        'spy.': 4,'multihop.': 4,'named.': 4,'snmpguess.': 4,'worm.': 4,'snmpgetattack.': 4,
        'xsnoop.': 4,'xlock.': 4,'sendmail.': 4,
        'normal.': 0}
    
    continuous_features=["duration","src_bytes",
    "dst_bytes","wrong_fragment","urgent","hot","num_failed_logins","num_compromised","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate"]


    
    #Changing the class labels
    X['class_label_main'] = X['class_label'].apply(lambda i: mapping[i])
   

    X=X.drop(['class_label'], axis = 1)
    
    
    #LabelEncoder() function Encode target labels with value between 0 and n_classes-1
    le = LabelEncoder()

    # protocol
    X_protocol_type = le.fit_transform(X['protocol_type'].values)
    X_service = le.fit_transform(X['service'].values)
    X_flag = le.fit_transform(X['flag'].values)

    X_main = X[continuous_features]
    
    X_main['protocol_type'] = X_protocol_type
    X_main['service'] = X_service
    X_main['flag'] = X_flag
    
    #y_main=X['class_label_main']
    
    
    return X_main #,y_main

In [4]:
#X,y=data_preprocessing(df_test)

#X.head()

In [5]:
#X['class_label_prd'] = y

#X['class_label_org'] = df_test['class_label']

#X.head()

In [6]:
#X.to_csv('final_test_for_deploy.csv')

In [7]:
def final(X):
    
    X=data_preprocessing(X)
    #X=X.drop('class_label_main', axis=1)
    
    binary_model = joblib.load('C:\\Users\\91912\\OneDrive\\Desktop\\cs1\\Trained_model\\RandomForestClassifier_over_sampling_SMOTETomek.joblib')
    binary_model_pred = binary_model.predict(X)
    
    multiclass_model = joblib.load('C:\\Users\\91912\\OneDrive\\Desktop\\cs1\\Trained_model\\XGBClassifier_multiclass_without_sampling.joblib')
    multiclass_model_pred = multiclass_model.predict(X)
    
    i = [0 if j == 0 else 1 for j in binary_model_pred]
    j=int(i[0])
    
    k = [l for l in multiclass_model_pred]   
    l=int(k[0]) 

    
    if j == 1:
        
        
        print("The Result in Instrusion Singal", l)
    
    else:
        
        print("The Result in Non Instrusion Singal", j)

#### Predicted Result 1

In [8]:
d=df_test.sample()
print(d['class_label'])

47609    neptune.
Name: class_label, dtype: object


In [9]:
final(d)

The Result in Instrusion Singal 2


#### Predicted Result 2

In [10]:
d=df_test.sample()
print(d['class_label'])

19945    normal.
Name: class_label, dtype: object


In [11]:
final(d)

The Result in Non Instrusion Singal 0


#### Predicted Result 3

In [19]:
d=df_test.sample()
print(d['class_label'])

65577    neptune.
Name: class_label, dtype: object


In [20]:
final(d)

The Result in Instrusion Singal 2


#### Predicted Result 4

In [28]:
d=df_test.sample()
print(d['class_label'])

21680    neptune.
Name: class_label, dtype: object


In [29]:
final(d)

The Result in Instrusion Singal 2


#### Predicted Result 5

In [39]:
d=df_test.sample()
print(d['class_label'])

64086    neptune.
Name: class_label, dtype: object


In [40]:
final(d)

The Result in Instrusion Singal 2


#### Predicted Result 6

In [51]:
d=df_test.sample()
print(d['class_label'])

9824    satan.
Name: class_label, dtype: object


In [52]:
final(d)

The Result in Instrusion Singal 1


#### Predicted Result 7

In [53]:
d=df_test.sample()
print(d['class_label'])

59241    normal.
Name: class_label, dtype: object


In [54]:
final(d)

The Result in Non Instrusion Singal 0


#### Predicted Result 8

In [55]:
d=df_test.sample()
print(d['class_label'])

8142    normal.
Name: class_label, dtype: object


In [56]:
final(d)

The Result in Non Instrusion Singal 0
